#### Link of the Pycaret docs : https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [1]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo de PIB
df = pd.read_excel('PIB_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date   etanol         PIB
0  2017   412770  2063514.69
1  2018   720236  1916933.71
2  2019  1330163  1881458.56
3  2020  2430153  1448565.67
4  2021  3296386  1608981.30
5  2022  4141401  1445811.00
6  2023  4986399  1376939.00
7  2024  5831395  1308067.00
8  2025  6676391  1239196.00
9  2026  7521387  1170324.00


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    14 non-null     int64  
 1   etanol  14 non-null     int64  
 2   PIB     14 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 464.0 bytes


### Avaliando a produção de Etanol

In [4]:
df.drop(['etanol'], axis = 1,inplace=True)
#df = df[df['tipo']=='soja']
df.head(10)


,date,PIB
0,2017,2063514.69
1,2018,1916933.71
2,2019,1881458.56
3,2020,1448565.67
4,2021,1608981.30
5,2022,1445811.00
6,2023,1376939.00
7,2024,1308067.00
8,2025,1239196.00
9,2026,1170324.00


In [5]:
df.set_index('date',inplace=True)

In [6]:
df=df.sort_index(axis=0)
print(df.head(10))

             PIB
date            
2017  2063514.69
2018  1916933.71
2019  1881458.56
2020  1448565.67
2021  1608981.30
2022  1445811.00
2023  1376939.00
2024  1308067.00
2025  1239196.00
2026  1170324.00


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [7]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,PIB
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(14, 1)"
5,Transformed data shape,"(14, 1)"
6,Transformed train set shape,"(13, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [8]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Checking Train-Test Split

In [ ]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [ ]:
top4 = exp.compare_models(n_select = 4)

## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [ ]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})

###  huber_cds_dt    Huber w/ Cond. Deseasonalize & Detrending

In [ ]:
en_cds_dt = exp.create_model('en_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = en_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## lightgbm_cds_dt Light Gradient Boosting w/ Cond. Deseasonalize & Detrending

In [ ]:
ets  = exp.create_model('ets')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(ets, fh = 9)


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2022,1.416451e+06
2023,1.314730e+06
2024,1.213008e+06
2025,1.111287e+06
2026,1.009566e+06
2027,9.078446e+05
2028,8.061234e+05
2029,7.044021e+05
2030,6.026809e+05


## polytrend   Polynomial Trend Forecaster

In [ ]:

polytrend = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = polytrend, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


#### theta   Theta Forecaster

In [ ]:

theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 9},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(theta, fh = 9)

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2022,1.445811e+06
2023,1.376939e+06
2024,1.308067e+06
2025,1.239196e+06
2026,1.170324e+06
2027,1.101452e+06
2028,1.032580e+06
2029,9.637086e+05
2030,8.948369e+05


## ALL GRAPHS

In [ ]:
exp.plot_model(estimator = en_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

               

NameError: name 'huber_cds_dt' is not defined